# Food Delivery Dataset Merger

This notebook merges three data sources:
- **orders.csv**: Order details with restaurant names
- **users.json**: User information (name, city, membership)
- **restaurants.sql**: Restaurant details (cuisine, rating)

In [1]:
import pandas as pd
import sqlite3
import os

## Load Data Sources

In [2]:
# Get the directory where this notebook is located
script_dir = os.path.dirname(os.path.abspath('__file__'))

# Load CSV file
orders = pd.read_csv(os.path.join(script_dir, 'orders.csv'))
print(f"Orders loaded: {len(orders)} rows")
print(f"Columns: {list(orders.columns)}")
orders.head()

Orders loaded: 10000 rows
Columns: ['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name']


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
# Load JSON file
users = pd.read_json(os.path.join(script_dir, 'users.json'))
print(f"Users loaded: {len(users)} rows")
print(f"Columns: {list(users.columns)}")
users.head()

Users loaded: 3000 rows
Columns: ['user_id', 'name', 'city', 'membership']


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [ ]:
# Load SQL file into sqlite
conn = sqlite3.connect(':memory:')
with open(os.path.join(script_dir, 'restaurants.sql'), 'r') as f:
    conn.executescript(f.read())

restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)
print(f"Restaurants loaded: {len(restaurants)} rows")
print(f"Columns: {list(restaurants.columns)}")
restaurants.head()

## Merge Datasets

Using left joins to preserve all order records:
1. Merge orders with users on `user_id`
2. Merge result with restaurants on `restaurant_id`

In [ ]:
# Merge datasets
merged = orders.merge(users, on='user_id', how='left') \
               .merge(restaurants, on='restaurant_id', how='left')

print(f"Merged dataset: {len(merged)} rows")
print(f"Columns: {list(merged.columns)}")
merged.head()

## Save Final Dataset

In [ ]:
# Save final dataset
output_path = os.path.join(script_dir, "final_food_delivery_dataset.csv")
merged.to_csv(output_path, index=False)

print(f"Dataset successfully merged and saved to: {output_path}")
print(f"\nDataset Info:")
merged.info()